In [1]:
# third parties module
import pandas as pd
import numpy as np
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation

# local module
from utils import load_data

In [3]:
# load data
data = load_data ('text')
# show some data
print ('\n'.join (np.unique (data['cluster'])))
print (data.head (5))

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc
                                                text                cluster
0  I was wondering if anyone out there could enli...              rec.autos
1  A fair number of brave souls who upgraded thei...  comp.sys.mac.hardware
2  well folks, my mac plus finally gave up the gh...  comp.sys.mac.hardware
3  \nDo you have Weitek's address/phone number?  ...          comp.graphics
4  From article <C5owCB.n3p@world.std.com>, by to...              sci.space


In [4]:
# remove stop word and basic preprocessing
stopword = nltk.corpus.stopwords.words ('english')
punctuation = string.punctuation + '@<?'
def preprocessing (text):
    # first lowering
    text = text.lower ()
    # replace punctuation
    for punc in punctuation:
        text.replace (punc, ' ')
    # replace stopwords
    words = nltk.word_tokenize (text)
    words = [w for w in words if w not in stopword and len (w) >= 3]
    
    return ' '.join (words)

data['token'] = data['text'].apply (preprocessing)    

In [5]:
# TF-IDF top 1000 --> normalize
vectorizer = TfidfVectorizer (max_features=1000)
features = vectorizer.fit_transform (data['token'])
print (features.shape)

(11314, 1000)


In [6]:
num_topics = 15
lda = LatentDirichletAllocation (num_topics)
lda.fit (features)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=15, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)

In [7]:
mapped_data = lda.transform (features)

In [8]:
sorted_topic = lda.components_.argsort ()[:,::-1]
terms = vectorizer.get_feature_names ()
for k in range (num_topics):
    label = []
    for t_idx in sorted_topic[k,:5]:
        label.append (terms[t_idx])
    print ("Topic {}, centroid : {}".format (k, ' '.join (label)))

Topic 0, centroid : edu soon 800 number answer
Topic 1, centroid : card monitor video thanks anyone
Topic 2, centroid : space nasa launch test earth
Topic 3, centroid : sale 00 offer shipping condition
Topic 4, centroid : book books information published reference
Topic 5, centroid : deleted stuff ax source sites
Topic 6, centroid : drive scsi drives disk controller
Topic 7, centroid : god people one would think
Topic 8, centroid : game team year games players
Topic 9, centroid : windows use file program files
Topic 10, centroid : edu david mit university clinton
Topic 11, centroid : car one bike like would
Topic 12, centroid : edu ftp 1993 pub site
Topic 13, centroid : please thanks mail com anyone
Topic 14, centroid : would people government one gun


In [9]:
idx = 31
print (mapped_data[idx])
print ()
print (sum (mapped_data[idx]))
print ()
print (data.loc[idx, 'token'])
print (data.loc[idx, 'cluster'])

[0.01014303 0.01014303 0.01014302 0.01014308 0.01014303 0.01014302
 0.01014302 0.04163018 0.01014303 0.01014304 0.01014302 0.82651003
 0.01014306 0.01014304 0.01014337]

1.0

14-y-o son usual teenage spotty chin greasy nose bought clearasil face wash ointment think probably enough along usual good diet however get product called dalacin used doctor's-prescription treatment available chemist counter asked couple pharmacists say either acne severe enough dalacin clearasil ok. odd spots teenager nothing serious father n't figure acne going escalate something disfiguring know kids senstitive appearance wary neighbour son wierd malady eventually put overdose vitamin acne treatment want help appropriate treatment son also scaliness around hairline scalp sort teenage cradle cap pointers/advice tried couple anti dandruff shampoos inclined make condition worse better shall bury kid till
sci.med
